In [169]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC #selenium에서 사용할 모듈 import
from selenium.webdriver import ActionChains

import pandas as pd
import bs4
import time
import requests
from bs4 import BeautifulSoup
import re
import csv

########라이브러리 정의

options = webdriver.ChromeOptions()
driver_loc = "../externlib/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(driver_loc , options=options) #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정

driver.get("https://map.naver.com/v5/") #네이버 신 지도 
try:
    element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
   ) #입력창이 뜰 때까지 대기
finally:
    pass

name = "none"
menu = "none"
menuList=[]

reviewInfo = "none" # 별점 방문자 블로그 리뷰수 변수

divideAgeRatioStringList="none"
tmpEachAgeRatioList = []



###########사용하는변수 
targetName = "none" #가게이름
targetReviewsNumber = "none" #가게 블로그리뷰, 방문자 리뷰수
score = "none" #별점
blogCount = "none" # 블로그 리뷰수
visitCount = "none" # 방문자 리뷰수
tenCen = "none" #10대 인기도
twoCen="none" # 20대 인기도 
threeCen = "none" #30대 인기도
fourCen = "none" #40대 인기도
fiveCen ="none" #50대 인기도
sixCen = "none" #60대 인기도
womanRatio="none" # 여성 검색 인기도
manRatio="none"   #남성 검색 인기도
############데이터 프레임에 넣을 변수


search_box = driver.find_element_by_class_name("input_search")
search_box.send_keys("건대 음식점")
search_box.send_keys(Keys.ENTER) #검색창에 "서울 칵테일바" 입력

time.sleep(3) #화면 표시 기다리기

element = driver.find_element_by_id("searchIframe")

driver.switch_to.frame(element) #프레임 이동
time.sleep(4) #화면 표시 기다리기



scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[1]")
#검색 결과로 나타나는 scroll-bar 포함한 div 잡고
driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", scroll_div)
time.sleep(2)
#여기까지 scroll
#맨 아래까지 내려서 해당 페이지의 내용이 다 표시되게 함



selectWeb = driver.find_elements_by_class_name("O9Z-o")
# 50개 선택창 아이프레임 들어갔다 나오기

C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/1759181434.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_loc , options=options) #selenium 사용에 필요한 chromedriver.exe 파일 경로 지정
C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/1759181434.py:58: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  search_box = driver.find_element_by_class_name("input_search")
C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/1759181434.py:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("searchIframe")
C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/1759181434.py:71: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[

In [178]:
menuList =[]

finalResult = []

In [180]:
for i in range(0, len(selectWeb)):
# try:
    time.sleep(2)
    # 이쪽 페이지 있는거 포문으로 돌기
    ##검색된 거 하나하나 들어가는 곳
    selectWeb[i].click()
    time.sleep(3)

    driver.switch_to.default_content()  #프레임 빠져나오기

    element = driver.find_element_by_id("entryIframe")

    driver.switch_to.frame(element) #프레임 이동
    # 개별정보 접근시 프레임 이동
    time.sleep(2)
    ##현재 페이지 소스 가져오기

    html = driver.page_source
    time.sleep(2)
    ## BeautifulSoup로 페이지 소스 파싱
    bs = bs4.BeautifulSoup(html, "html.parser")
    time.sleep(2)


    name = bs.find(attrs={"class":"_3XamX"}).text
    try:
        categoryInfoList = driver.find_element_by_class_name("_3ocDE")

        gubun = categoryInfoList.text

    except Exception as e:
        gubun = "none"


    time.sleep(2)

    # 가게 이름
    sideInfoList = driver.find_elements_by_class_name("_20Ivz")
    categoryInfoList = driver.find_elements_by_class_name("_3ocDE")
    step1 = sideInfoList[0].text.split("방문자리뷰")
    checkPoint = step1[0]

    try:
        if(checkPoint==""):
            pass
        else:
            sideInfoList = driver.find_elements_by_class_name("_20Ivz")
            time.sleep(2)
            step1 = sideInfoList[0].text.split("방문자리뷰")
            time.sleep(2)
            score = step1[0].split("\n")[1]
            time.sleep(2)
    except Exception as e:
        print(e)
    # 별점 있는것과 없는것 따로 구분
    try:
        time.sleep(2)
        countAmount = driver.find_elements_by_class_name("place_section")
        time.sleep(2)

        blogCount = "블로그 리뷰"+"_"+countAmount[0].text.split("블로그리뷰")[1].split("\n")[0]
        visitCount = "방문자 리뷰"+"_"+ countAmount[0].text.split("블로그리뷰")[0].split("방문자리뷰")[-1]

    except Exception as e:
        print(e)
        time.sleep(2)

    driver.switch_to.default_content()  #프레임 빠져나오기

    element = driver.find_element_by_id("searchIframe")
    driver.switch_to.frame(element) #프레임 이동

    tmpList = [name,gubun,score,visitCount,blogCount]
    finalResult.append(tmpList)

    checkList = [name]
    menuList.append(checkList)

C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/2447647361.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("entryIframe")
C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/2447647361.py:27: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  categoryInfoList = driver.find_element_by_class_name("_3ocDE")
C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/2447647361.py:41: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  sideInfoList = driver.find_elements_by_class_name("_20Ivz")
C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/2447647361.py:42: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  categoryInfoList = driver.find_elements_by_class_name("_3ocDE")
C:\Us

In [174]:
selectWeb = driver.find_elements_by_class_name("O9Z-o")

C:\Users\SMART18\AppData\Local\Temp/ipykernel_9680/392707679.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  selectWeb = driver.find_elements_by_class_name("O9Z-o")


In [175]:
selectWeb

[<selenium.webdriver.remote.webelement.WebElement (session="d8c2006e228176b662f6fd2fd008feee", element="26b892ff-6809-4ffc-8fc7-8baa86e9d378")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8c2006e228176b662f6fd2fd008feee", element="992ff0b8-25a8-4994-bb37-b40587ec53c5")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8c2006e228176b662f6fd2fd008feee", element="5ad7cd26-4ffe-453a-b754-fe66cf25a911")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8c2006e228176b662f6fd2fd008feee", element="9272a1af-0d96-4ea4-a1aa-689614f2dfff")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8c2006e228176b662f6fd2fd008feee", element="ddffcf0e-4556-4894-974f-3c886c410dba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8c2006e228176b662f6fd2fd008feee", element="837cb5c6-1634-43b6-9bec-fe63e3a68503")>,
 <selenium.webdriver.remote.webelement.WebElement (session="d8c2006e228176b662f6fd2fd008feee", element="4d238c85-1348-4b5a-8cfe-74

In [176]:
len(selectWeb)

50

In [177]:
selectWeb[0].text

'선남sea'

In [165]:
# 확인후 데이터 프레임으로 저장

In [184]:
columns = ["name","gubun","score","visitCount","blogCount"]

In [185]:
sinsa_age_gender = pd.DataFrame(data = finalResult,columns = columns)

In [187]:
sinsa_age_gender.to_csv("gundae6_score_final6.csv",mode="w", encoding = "ms949")